In [ ]:
import pandas as pd
from math import sqrt
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error
from numpy import append
import seaborn as sns 
import pandas as pd
from math import sqrt
import statistics
from numpy import asarray
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.metrics import mean_squared_error, mean_absolute_error
from matplotlib import pyplot
from numpy import zeros
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
import statsmodels
import numpy as np

In [ ]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [ ]:
feature_list = ['Random Forest', 'Bagging', 'Extra Trees', 'Linear Regressor', 'Elastic Net', 'Ridge', 'Lasso', 'Decision Tree', 'K Neighnours', 'Vector Auto Regression', 'VARMAX']
zero_data = zeros(shape=(4,len(feature_list)))
errors = pd.DataFrame(zero_data, columns=feature_list)
keys =['MAE', 'MSE', 'RMSE', 'MSLE']
errors.rename(index={0:keys[0],1:keys[1], 2:keys[2], 3:keys[3]}, inplace=True)
errors

In [ ]:
data1 = read_csv('/content/NO2 Data Final.csv')

In [ ]:
date = data1['date']
data1 = data1.drop(['date'], axis=1)
cols_o = data1.columns
data1['date']=date

In [ ]:
data1 = data1.drop(['date'], axis =1)

In [ ]:
dates_May_Apr = asarray(date[len(date)-61:])

In [ ]:
dates_May_Apr

In [ ]:
#data1 = data1.drop(['Unnamed: 0'], axis=1)
values = data1.values

# **Statistical Forecasting Models**# 


1.   Vector Auto Regression
2.   Vector Autoregression Moving-Average


In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.varmax import VARMAX

In [ ]:
train = data1[:int(0.8*(len(data1)))]
valid = data1[int(0.8*(len(data1))):]
cols = data1.columns
history = asarray(train)
prediction = []
#fit the model

for i in range(len(valid)):
    model = VAR(endog=history)
    model_fit = model.fit()
    prediction_= model_fit.forecast(model_fit.y, 1)
    prediction.append(prediction_)
    valid_one = asarray(valid.iloc[i]).reshape(1,64)
    history = append(history, valid_one, axis = 0)
# make prediction on validation
    

In [ ]:
prediction = asarray(prediction)
prediction = prediction.reshape(len(valid),64)

In [ ]:
pred = asarray(prediction)
valid = asarray(valid)
mse = mean_squared_error(pred, valid)
mae = mean_absolute_error(pred, valid)
mape = mean_absolute_percentage_error(valid,pred)
rmse = sqrt(mse)

In [ ]:
name = 'VARMAX'
print(name,  ':Mean Absolute Error (Nitrogen Dioxide):', mae)
print(name,  ':Mean Squared Error (Nitorgen Dioxide):', mse)
print(name,  ':Root Mean Squared Error (Nitrogen Dioxide):', rmse)
print(name,  ':Mean Absolue Percentage Error (Nitrogen Dioxide):', mape)

errors[name]['MAE']=mae
errors[name]['MSE']=mse
errors[name]['RMSE']=rmse
errors[name]['MLSE']=0

predic = pred[11,:]
predic = predic.reshape(8,8)
predic = predic.astype('float')

import matplotlib.pyplot as plt

test = valid[11,:]
test = test.reshape(8,8)
test = test.astype(float)

plt.subplot(1, 2, 1)
plt.imshow(test)
plt.title('Expected')
plt.subplot(1, 2, 2)
plt.imshow(predic)
plt.title('Predicted')

#fig.subplots_adjust(right=0.8)
cbar = plt.colorbar()
cbar.set_label('Nitrogen Dioxide (ug/m^3)',size=8)
plt.show()


plt.plot(dates_May_Apr, valid[len(valid)-61:,54], label='Expected')
plt.plot(dates_May_Apr,pred[len(pred)-61:,54], label='Predicted')
plt.xticks(rotation=90)
plt.xlabel('Date')
plt.ylabel('Nitrogen Dioxide (ug/m^3)')
plt.legend()
plt.title('VECTOR AUTO REGRESSION')

fig = plt.gcf()
fig.set_size_inches(25, 5)
fig.savefig('test2png.png', dpi=100)

# **Data Preparation for Supervised Learning Problem**

In [ ]:
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [ ]:
len(values)

In [ ]:
data = series_to_supervised(values, 30, 15)
cols = data.columns
len(cols)-64

In [ ]:
cols = data.columns
from sklearn.base import clone

In [ ]:
def predictions_no_walk(o_data, data, name, index, model):
  predictions = []
  #train = data.iloc[0:int(0.80*len(data))]
  #test = data.iloc[int(0.80*len(data)):]
  x_data = data.iloc[:][cols[0:len(cols)-15*64]]
  y_data = data.iloc[:][cols[len(cols)-15*64:]]
  
  model = clone(model)
  #model.fit(x_data, y_data)
  #model.fit(train_x_a, train_y_a)
  #history = train
	# step over each time-step in the test set
  for i in range(2):

    model.fit(x_data, y_data)
    yhat = model.predict([x_data.iloc[-1]])
    yhat = yhat.reshape(15,64)
    o_data = append(o_data, yhat, axis=0)
    #o_data.append(yhat)
    u_data = series_to_supervised(o_data, 30, 15)
    x_data = u_data.iloc[:][cols[0:len(cols)-15*64]]
    y_data = u_data.iloc[:][cols[len(cols)-15*64:]]
		# store forecast in list of predictions
    predictions.append(yhat)

  
  predictions = asarray(predictions)
  predictions = predictions.reshape(2,64*15)
  predic_64 =zeros((1,64))
  #predic_64 = asarray(predic_64)
  for i in range(0,len(predictions)):
     pred_temp = predictions[i].reshape(15,64)
     predic_64 = append(predic_64, pred_temp, axis=0)
  #mae = asarray(test_y)
  #error = mean_absolute_error(np.asrray(test_y), predictions)
  pred = DataFrame(predic_64)
  return  predic_64

In [ ]:
# walk-forward validation for univariate data
def walk_forward_validation(data, name,index, model):
  predictions = []
  train = data.iloc[0:int(0.80*len(data))]
  test = data.iloc[int(0.80*len(data)):]
  x_data = data.iloc[:][cols[0:192]]
  y_data = data.iloc[:][cols[192:]]
  
  test_x = x_data.iloc[int(0.80*len(x_data)):]
  test_y = y_data.iloc[int(0.80*len(y_data)):]

  #train_x = train.iloc[:][cols[0:192]]
  #train_y = train.iloc[:][cols[192:]]
  #train_x_a= asarray(train_x)
  #train_y_a = asarray(train_y)
  model = clone(model)
  #model.fit(train_x_a, train_y_a)
  history = train
	# step over each time-step in the test set
  for i in range(len(test)):
    testX, testy = test_x.iloc[i], test_y.iloc[i]
		# fit model on history and make a prediction
    train_x = history.iloc[:][cols[0:192]]
    train_y = history.iloc[:][cols[192:]]
    train_x_a= asarray(train_x)
    train_y_a = asarray(train_y)
    model.fit(train_x_a, train_y_a)
    yhat = model.predict([testX])
		# store forecast in list of predictions
    predictions.append(yhat)
		# add actual observation to history for the next loop
    history.append(test.iloc[i])
		# summarize progress
    #print('Predicted:', asarray(yhat), 'Expected:', asarray(test_y.iloc[i]))
	# estimate prediction error
  
  predictions = asarray(predictions)
  predictions = predictions.reshape(len(test),64)
  mae = asarray(test_y)
  #error = mean_absolute_error(np.asrray(test_y), predictions)
  pred = DataFrame(predictions)
  pred.to_csv('NO2_predictions.csv')

  maes = mean_absolute_error(mae, predictions)
  mse = mean_squared_error(mae, predictions)
  rmse = sqrt(mse)
  mape = mean_absolute_percentage_error(mae,pred)
  if name != 'Linear Regressor' and name!= 'Ridge CV':
      msle = mean_squared_log_error(predictions, mae)
  else:
      msle = None

  print(name,  ':Mean Absolute Error (Nitrogen Dioxide):', maes)
  print(name,  ':Mean Squared Error (Nitrogen Dioxide):', mse)
  print(name,  ':Root Mean Squared Error (Nitrogen Dioxide):', rmse)
  print(name,  ':Mean Absolute Percentage Error (Nitrogen Dioxide):', mape)

  if name != 'Linear Regressor' and name!= 'Ridge CV':
      print(name,  ':Mean Squared Logarithmic Error (Nitrogen Dioxide):', msle)

  expected = mae[index].reshape(8,8)
  predicted = predictions[index].reshape(8,8)

  import matplotlib.pyplot as plt

  plt.subplot(1, 2, 1)
  plt.imshow(expected)
  plt.title('Expected')
  plt.subplot(1, 2, 2)
  plt.imshow(predicted)
  plt.title('Predicted')
  plt.show()
  return  mape,maes, mse, rmse, msle, predictions, mae

In [ ]:
# walk-forward validation for univariate data
def walk_forward_validation(data, name, index, model):
  predictions = []
  train = data.iloc[0:int(0.998*len(data))]
  test = data.iloc[int(0.998*len(data)):]
  x_data = data.iloc[:][cols[0:len(cols)-15*64]]
  y_data = data.iloc[:][cols[len(cols)-15*64:]]
  #cols[0:len(cols)-15*64]
  test_x = x_data.iloc[int(0.998*len(x_data)):]
  test_y = y_data.iloc[int(0.998*len(y_data)):]

  #train_x = train.iloc[:][cols[0:192]]
  #train_y = train.iloc[:][cols[192:]]
  #train_x_a= asarray(train_x)
  #train_y_a = asarray(train_y)
  model = clone(model)
  #model.fit(train_x_a, train_y_a)
  history = train
	# step over each time-step in the test set
  for i in range(len(test)):
    testX, testy = test_x.iloc[i], test_y.iloc[i]
		# fit model on history and make a prediction
    train_x = history.iloc[:][cols[0:len(cols)-15*64]]
    train_y = history.iloc[:][cols[len(cols)-15*64:]]
    train_x_a= asarray(train_x)
    train_y_a = asarray(train_y)
    model.fit(train_x_a, train_y_a)
    yhat = model.predict([testX])
		# store forecast in list of predictions
    predictions.append(yhat)
		# add actual observation to history for the next loop
    history.append(test.iloc[i])
		# summarize progress
    #print('Predicted:', asarray(yhat), 'Expected:', asarray(test_y.iloc[i]))
	# estimate prediction error
  test_y = asarray(test_y)
  test_y = test_y.reshape(len(test_y), 15*64)
  predictions = asarray(predictions).reshape(len(predictions), 15*64)
  predic_64 =zeros((1,64))
  y_64=zeros((1,64))
  for i in range(0,len(predictions)):
     pred_temp = predictions[i].reshape(15,64)
     y_temp = test_y[i].reshape(15,64)
     predic_64 = append(predic_64, pred_temp, axis=0)
     y_64=append(y_64, y_temp, axis=0)
  #predictions = asarray(predictions)
  #predictions = predictions.reshape(len(test),64)
  
  #error = mean_absolute_error(np.asrray(test_y), predictions)
  pred = DataFrame(predic_64)
  pred.to_csv('NO2_predictions.csv')
  
  maes = mean_absolute_error(test_y, predictions)
  mse = mean_squared_error(test_y, predictions)
  rmse = sqrt(mse)
  mape = mean_absolute_percentage_error(test_y,predictions)
  if name != 'Linear Regressor' and name!= 'Ridge CV':
      msle = mean_squared_log_error(predictions, test_y)
  else:
      msle = None

  print(name,  ':Mean Absolute Error (Nitrogen Dioxide):', maes)
  print(name,  ':Mean Squared Error (Nitrogen Dioxide):', mse)
  print(name,  ':Root Mean Squared Error (Nitrogen Dioxide):', rmse)
  print(name,  ':Mean Absolute Percentage Error (Nitrogen Dioxide):', mape)

  if name != 'Linear Regressor' and name!= 'Ridge CV':
      print(name,  ':Mean Squared Logarithmic Error (Nitrogen Dioxide):', msle)

  #expected = mae[index].reshape(8,8)
  #predicted = predictions[index].reshape(8,8)

  import matplotlib.pyplot as plt

  #plt.subplot(1, 2, 1)
  #plt.imshow(expected)
  #plt.title('Expected')
  #plt.subplot(1, 2, 2)
  #plt.imshow(predicted)
  #plt.title('Predicted')
  #plt.show()
  return  mape, maes, mse, rmse, msle, predic_64, y_64

# **Ensemble Models**

1.   Random Forest Regressor
2.   Bagging Regressor
3.   Extra Trees Regressor


In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
June_date = []
year = '2021'
month = '6'
for i in range(1,32):
  d = month + '/' + str(i) +'/' + year
  June_date.append(d)

In [ ]:
total_dates = append(asarray(date), June_date)

In [ ]:
date_3_months = total_dates[len(total_dates)-92:]

In [ ]:
def predictions_3_month (predictions_June, predictions, date_3_months, cols_o, name,i):
    Apr_May_prediction = predictions[len(predictions)-61:,:]
    Apr_May_June_pred = append(Apr_May_prediction, predictions_June, axis=0)
    Apr_May_June_NO2_pred = DataFrame(Apr_May_June_pred, columns = [cols_o])
    Apr_May_June_NO2_pred['date'] = date_3_months[0:len(date_3_months)-1]
    Apr_May_June_NO2_a = asarray(Apr_May_June_NO2_pred)
    pyplot.plot(date_3_months[0:len(date_3_months)-16], Apr_May_June_NO2_a[0:len(Apr_May_June_NO2_a)-15 ,i], marker='o', ms=4, label='Predicted')
    pyplot.plot(dates_May_Apr[0:len(dates_May_Apr)-15], y_a[len(y_a)-46:,i], marker='o', ms=4,label='Expected')
    pyplot.xticks(fontsize=13, rotation=90)
    pyplot.xlabel('Date')
    pyplot.ylabel('Nitrogen Dioxide (ug/m^3)')
    pyplot.legend(fontsize=15)
    pyplot.title(name, fontdict={'fontsize': 20, 'fontweight': 'medium'})

    axes = pyplot.gca()
    axes.xaxis.label.set_size(15)
    axes.yaxis.label.set_size(15)

    fig = pyplot.gcf()
    fig.set_size_inches(23, 5)
    return Apr_May_June_NO2_pred

In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y_a = walk_forward_validation(data_x, 'Extra Trees Regressor', index = 249, model = ExtraTreesRegressor(n_estimators=24, max_depth=18))
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model  = ExtraTreesRegressor(n_estimators=24, max_depth=18))



In [ ]:
predictions_extra = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o,'Extra Trees', i=30)
errors['Extra Trees']['MAE']=mae
errors['Extra Trees']['MSE']=mse
errors['Extra Trees']['RMSE']=rmse
errors['Extra Trees']['MLSE']=msle

In [ ]:
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model  = ExtraTreesRegressor(n_estimators=24, max_depth=18))


In [ ]:
predictions_extra = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'Extra Trees Regressor')
predictions_extra.to_csv('predictions_3_months_extra_trees.csv')

In [ ]:
values.shape

In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y_a = walk_forward_validation(data_x, 'Bagging Regressor', index = 298, model = BaggingRegressor(n_estimators=2 ))
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Bagging Regressor', index = 249, model = BaggingRegressor(n_estimators=2))
predictions_bagging = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'Bagging Regressor', 30)
predictions_bagging.to_csv('NO2_predictions_June_bagging.csv')
errors['Bagging']['MAE']=mae
errors['Bagging']['MSE']=mse
errors['Bagging']['RMSE']=rmse
errors['Bagging']['MLSE']=msle


In [ ]:
predictions_June = predictions_no_walk(values, data, 'Bagging Regressor', index = 249, model = BaggingRegressor(n_estimators=10))
predictions_bagging = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'Bagging Regressor')
predictions_bagging.to_csv('NO2_predictions_June_bagging.csv')
errors['Bagging']['MAE']=mae
errors['Bagging']['MSE']=mse
errors['Bagging']['RMSE']=rmse
errors['Bagging']['MLSE']=msle

In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y = walk_forward_validation(data_x, 'Random Forest Regressor', index = 297, model=RandomForestRegressor(n_estimators=2))


In [ ]:
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Random Forest Regressor', index = 249, model = RandomForestRegressor(n_estimators=2))


In [ ]:
predictions_RF = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'Random Forest', i = 30)
predictions_RF.to_csv('NO2_predictions_June_Random_Forest.csv')
errors['Random Forest']['MAE']=mae
errors['Random Forest']['MSE']=mse
errors['Random Forest']['RMSE']=rmse
errors['Random Forest']['MLSE']=msle

# **Linear Models** 

1.   Linear Regression
2.   Elastic Net Regression
3.   Ridge Linear Regression
4.   Lasso Linear Regression


In [ ]:
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV

In [ ]:
#data_x = series_to_supervised(values, 3, 1)
#cols = data_x.columns
data = series_to_supervised(values, 30, 15)
cols = data.columns
mape, mae, mse, rmse, msle, predictions, y_a = walk_forward_validation(data, 'Linear Regressor', index = 200, model = LinearRegression())
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model = LinearRegression())
predictions_linear = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'Linear Regressor', 30)
predictions_linear.to_csv('NO2_predictions_June_Linear_Regression.csv')
errors['Linear Regressor']['MAE']=mae
errors['Linear Regressor']['MSE']=mse
errors['Linear Regressor']['RMSE']=rmse
errors['Linear Regressor']['MLSE']=msle

#fig.savefig('test2png.png', dpi=100)

In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y = walk_forward_validation(data_x, 'Lasso', index = 300, model = Lasso())
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model = Lasso())
predictions_lasso = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'Lasso', i =30)
errors['Lasso']['MAE']=mae
errors['Lasso']['MSE']=mse
errors['Lasso']['RMSE']=rmse
errors['Lasso']['MLSE']=msle

In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y = walk_forward_validation(data_x, 'Elastic Net', index = 300, model = ElasticNet())
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model = ElasticNet())
predictions_elastic = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, name = 'Elastic Net', i =30)
errors['Elastic Net']['MAE']=mae
errors['Elastic Net']['MSE']=mse
errors['Elastic Net']['RMSE']=rmse
errors['Elastic Net']['MLSE']=msle


In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y = walk_forward_validation(data_x, 'Ridge CV', index = 160, model = RidgeCV())
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model = RidgeCV())
predictions_ridge = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o,'Ridge CV', 20)
errors['Ridge']['MAE']=mae
errors['Ridge']['MSE']=mse
errors['Ridge']['RMSE']=rmse
errors['Ridge']['MLSE']=msle


# **Non Linear Models**

1.   K Neighbour Regressor
2.   Decision Trees Regressor


In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y = walk_forward_validation(data_x, 'Decision Tree Regressor', index = 300, model = DecisionTreeRegressor( max_depth = 16))
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model = DecisionTreeRegressor( max_depth = 16))
predictions_decision_trees = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'Decision Tree', 30)
predictions_decision_trees.to_csv('predictions_three_months_decision_trees.csv')
errors['Decision Tree']['MAE']=mae
errors['Decision Tree']['MSE']=mse
errors['Decision Tree']['RMSE']=rmse
errors['Decision Tree']['MLSE']=msle

#predicted = predictions[:,3]
#expected = y[:,3]
#pyplot.plot(dates_May_Apr, y[len(predictions)-61:,3], label='Expected')
#pyplot.plot(dates_May_Apr, predictions[len(predictions)-61:,3], label='Predicted')
#plt.xticks(rotation=90)
#pyplot.xlabel('Date')
#pyplot.ylabel('Nitrogen Dioxide (ug/m^3)')
#pyplot.legend()

#fig.savefig('test2png.png', dpi=100)

In [ ]:
predicted = predictions[1:,17]
expected = y[1:,17]
pyplot.plot(predicted, label='Expected')
pyplot.plot(expected, label='Predicted')
plt.xticks(rotation=90)
pyplot.xlabel('Date')
pyplot.ylabel('Nitrogen Dioxide (ug/m^3)')
pyplot.legend()
pyplot.title('Decision Tree Regressor')



print(mean_absolute_percentage_error(y[len(predictions)-61:,9],predictions[len(predictions)-61:,9]))


In [ ]:
data_x = series_to_supervised(values, 30, 15)
cols = data_x.columns
mape, mae, mse, rmse, msle, predictions, y_a = walk_forward_validation(data_x, 'K Neighbour Regressor', index = 140, model = KNeighborsRegressor(n_neighbors=100))
data = series_to_supervised(values, 30, 15)
cols = data.columns
predictions_June = predictions_no_walk(values, data, 'Extra Trees Regressor', index = 249, model = KNeighborsRegressor(n_neighbors=100))
predictions_KNN = predictions_3_month (predictions_June[1:,:], predictions, date_3_months, cols_o, 'K Neighbour Regressor', 30)
errors['K Neighnours']['MAE']=mae
errors['K Neighnours']['MSE']=mse
errors['K Neighnours']['RMSE']=rmse
errors['K Neighnours']['MLSE']=msle


# **Deep Learning Models : CNN**

In [ ]:
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LeakyReLU

def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return asarray(X), asarray(y)

In [ ]:
X, Y = split_sequences(values, 30, 15)
print(X.shape, Y.shape)
X_train, y_train, x_test, y = X[0:int(0.98*len(X)),:,:],Y[0:int(0.98*len(X)),:,:],X[int(0.98*len(X)):,:,:],Y[int(0.98*len(X)):,:,:]

In [ ]:
print(X_train.shape, y_train.shape)
n_output = Y.shape[1] * Y.shape[2]
Y = Y.reshape((Y.shape[0], n_output))
# the dataset knows the number of features, e.g. 2
n_features = X.shape[2]
n_steps_in, n_steps_out = 30, 15
print(Y.shape)

In [ ]:
# define model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, input_shape=(n_steps_in, n_features)))
model.add(LeakyReLU(alpha=0.05))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(50))
model.add(LeakyReLU(alpha=0.05))
#model.add(Dense(10))
#model.add(LeakyReLU(alpha=0.05))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mae')
model.fit(X_train, y_train.reshape((y_train.shape[0], n_output)), epochs=700, verbose=1)

In [ ]:
# demonstrate prediction
predictions=[]
for i in range(0,len(x_test)):
  x_input = x_test[i,:,:]
  x_input = x_input.reshape((1, n_steps_in, n_features))
  yhat = model.predict(x_input, verbose=0)
  predictions.append(yhat)
predictions=asarray(predictions).reshape(len(y), y.shape[1], y.shape[2])
predictions = predictions.reshape(len(predictions)*15,64)
y = y.reshape(len(y)*15, 64)
predictions_June_cnn = []
june = x_test[len(x_test)-1,:,:]
for i in range(0,2):
  x_input = june
  x_input = x_input.reshape((1, n_steps_in, n_features))
  yhat = model.predict(x_input, verbose=0)
  june = append(x_test[len(x_test)-1,15:30,:],yhat.reshape(15,64), axis=0)
  predictions_June_cnn.append(yhat)

predictions_June_cnn = asarray(predictions_June_cnn)
predictions_June_1 = predictions_June_cnn[0,:,:].reshape(15,64)
predictions_June_2 = predictions_June_cnn[1,:,:].reshape(15,64)
predictions_june = append(predictions_June_1, predictions_June_2 , axis=0)

maes = mean_absolute_error(y, predictions)
mse = mean_squared_error(y, predictions)
rmse = sqrt(mse)
mape = mean_absolute_percentage_error(y,predictions)
name = 'CNN'
print(name,  ':Mean Absolute Error (Nitrogen Dioxide):', maes)
print(name,  ':Mean Squared Error (Nitrogen Dioxide):', mse)
print(name,  ':Root Mean Squared Error (Nitrogen Dioxide):', rmse)
print(name,  ':Mean Absolute Percentage Error (Nitrogen Dioxide):', mape)

In [ ]:
cnn_3_months =  predictions_3_month (predictions_june, predictions, date_3_months, cols_o, 'CNN', 30)

# **Multilayer Perceptron (MLP)**

In [ ]:
X, Y = split_sequences(values, 30, 15)
print(X.shape, Y.shape)
X_train, y_train, x_test, y = X[0:int(0.98*len(X)),:,:],Y[0:int(0.98*len(X)),:,:],X[int(0.98*len(X)):,:,:],Y[int(0.98*len(X)):,:,:]
n_input = X_train.shape[1] * X_train.shape[2]
x_t = X_train.reshape((X_train.shape[0], n_input))

In [ ]:
x_t.shape

In [ ]:
n_output = y_train.shape[1]*y_train.shape[2]
print(n_output)

In [ ]:
y_train = y_train.reshape(len(y_train), 15*64)

In [ ]:
y_train.shape

In [ ]:
model = Sequential()
model.add(Dense(150, input_dim=n_input))
model.add(LeakyReLU(alpha=0.04))
model.add(Dense(100, input_dim=n_input))
model.add(LeakyReLU(alpha=0.04))
model.add(Dense(100, input_dim=n_input))
model.add(LeakyReLU(alpha=0.04))
model.add(Dense(n_output))
model.compile(optimizer='adam', loss='mape')
model.fit(x_t, y_train, epochs=30, verbose=1)

In [ ]:
model.fit(x_t, y_train, epochs=300, verbose=1)

In [ ]:
# demonstrate prediction
X, Y = split_sequences(values, 30, 15)
print(X.shape, Y.shape)
X_train, y_train, x_test, y = X[0:int(0.98*len(X)),:,:],Y[0:int(0.98*len(X)),:,:],X[int(0.98*len(X)):,:,:],Y[int(0.98*len(X)):,:,:]
predictions=[]
for i in range(0,len(x_test)):
  x_input = x_test[i,:,:]
  x_input = x_input.reshape((1,n_input))
  yhat = model.predict(x_input, verbose=0)
  predictions.append(yhat)
predictions=asarray(predictions).reshape(len(y), y.shape[1], y.shape[2])
predictions = predictions.reshape(len(predictions)*15,64)
y = y.reshape(len(y)*15, 64)
predictions_June_cnn = []
june = x_test[len(x_test)-1,:,:]
for i in range(0,2):
  x_input = june
  x_input = x_input.reshape((1,n_input))
  yhat = model.predict(x_input, verbose=0)
  june = append(x_test[len(x_test)-1,15:30,:],yhat.reshape(15,64), axis=0)
  predictions_June_cnn.append(yhat)

predictions_June_cnn = asarray(predictions_June_cnn)
predictions_June_1 = predictions_June_cnn[0,:,:].reshape(15,64)
predictions_June_2 = predictions_June_cnn[1,:,:].reshape(15,64)
predictions_june = append(predictions_June_1, predictions_June_2 , axis=0)

maes = mean_absolute_error(y, predictions)
mse = mean_squared_error(y, predictions)
rmse = sqrt(mse)
mape = mean_absolute_percentage_error(y,predictions)
name = 'MLP'
print(name,  ':Mean Absolute Error (Nitrogen Dioxide):', maes)
print(name,  ':Mean Squared Error (Nitrogen Dioxide):', mse)
print(name,  ':Root Mean Squared Error (Nitrogen Dioxide):', rmse)
print(name,  ':Mean Absolute Percentage Error (Nitrogen Dioxide):', mape)

In [ ]:
mlp_3_months =  predictions_3_month (predictions_june, predictions, date_3_months, cols_o, 'MLP', 30)

# **ConvLSTMs**



In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy import array
import pandas as pd
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow import keras
from tensorflow.keras import layers
import keras
import tensorflow as tf
from matplotlib import pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
data = series_to_supervised(values, 30, 15)
cols = data.columns
data.shape

In [ ]:
data_x = data.iloc[:][cols[0:30*64]]
data_y = data.iloc[:][cols[30*64:]]
print(data_x.shape, data_y.shape)
data_x = asarray(data_x)
data_y = asarray(data_y)

In [ ]:
data_x_=[]
data_y_=[]
for i in range(0,len(data_x)):
  d = data_x[i].reshape(30,1,8,8)
  ya = data_y[i].reshape(15,1,8,8)
  data_x_.append(d)
  data_y_.append(ya)

In [ ]:
input_sequence = asarray(data_x_)
output_sequence = asarray(data_y_)

In [ ]:
train_x = input_sequence[0:int(0.8*(input_sequence.shape[0]))]
train_y = output_sequence[0:int(0.8*(input_sequence.shape[0]))]
train_y.shape
print(train_x.shape, train_y.shape)

In [ ]:
test_x = input_sequence[int(0.8*(input_sequence.shape[0])):]
test_y = output_sequence[int(0.8*(input_sequence.shape[0])):]

In [ ]:
seq = keras.Sequential(
    [   keras.Input(
            shape=(30,1,8,8)
        ),  
        layers.ConvLSTM2D(
            filters=1, kernel_size=(5, 5),recurrent_activation='hard_sigmoid',
                         kernel_initializer='glorot_uniform', padding="same", data_format = 'channels_first', return_sequences=True
        ),
        layers.LeakyReLU(alpha=0.3),
        #layers.BatchNormalization(),
        #layers.ConvLSTM2D(
        #    filters=1, kernel_size=(2, 2), recurrent_activation='hard_sigmoid',
        #                 kernel_initializer='glorot_uniform', padding="same", data_format = 'channels_first', return_sequences=True
        #),
        #layers.LeakyReLU(alpha=0.3),
        layers.BatchNormalization(),
        layers.Conv3D(
           filters=15, kernel_size=(3, 3, 3), padding="same", data_format = 'channels_first'
        ),
    ]
)
opt = keras.optimizers.Adam(learning_rate=0.01)
seq.compile(loss="mape", optimizer=opt, metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [ ]:
seq.summary()

In [ ]:
epochs = 150
history  = seq.fit(
    train_x[:],
    train_y[:],
    batch_size = 32,
    epochs=epochs,
    verbose=2,
    validation_split=0.2
)

In [ ]:
seq.save('/content/drive/My Drive/FYP Models/regression_so2/')

In [ ]:
convlstm_model = keras.models.load_model('/content/drive/My Drive/FYP Models/regression_so2/')

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Loss')
plt.ylabel('Loss')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['mean_absolute_error'])
plt.plot(history.history['val_mean_absolute_error'])
plt.title('Mean Squared Error: Nitrogen Dioxide')
plt.ylabel('Mean Squared Error')
plt.xlabel('epoch')
plt.legend(['train','val'], loc='upper left')
plt.show()

In [ ]:
predictions = []
for i in range(0,len(test_x)):
    pred = convlstm_model.predict(test_x[i].reshape(1,30,1,8,8))
    predictions.append(pred)

In [ ]:
predictions_J = []
june = test_x[-1]
for i in range(0,2):
    pred = convlstm_model.predict(june.reshape(1,30,1,8,8))
    pred = asarray(pred)
    pred = pred.reshape(15,1,8,8)
    june = append(test_x[len(test_x)-1,15:,:,:,:], pred, axis = 0)
    predictions_J.append(pred.reshape(15,64))

In [ ]:
predictions = asarray(predictions)
test_y.shape

In [ ]:
predictions_j = asarray(predictions_J).reshape(30,64)

In [ ]:
predictions = predictions.reshape(len(predictions)*15, 64)
#test_y = test_y.reshape(len(test_y)*15, 64)

In [ ]:
mae = mean_absolute_error(test_y, predictions)
mse = mean_squared_error(test_y, predictions)
rmse = sqrt(mse)
mape = mean_absolute_percentage_error(test_y, predictions)
print('Mean Absolute Error:', mae)
print('Mean Squared Error', mse)
print('Root Mean Squared Error', rmse)
print('MAPE:', mape)

In [ ]:
convlstm_3_months =  predictions_3_month (predictions_j, predictions, date_3_months, cols_o, 'ConvLSTMs', 30)